<a href="https://colab.research.google.com/github/Supershivam07/Website/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
from numpy import zeros
from numpy import ones
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import glob
from skimage.io import imsave
from numpy.random import randint
from keras.initializers import RandomNormal
from matplotlib import pyplot as plt
from IPython.display import display
from PIL import Image
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, LeakyReLU, Dropout, BatchNormalization, Concatenate
from tensorflow.keras.layers import Conv2DTranspose, Activation, MaxPooling2D
from tensorflow.keras.losses import BinaryCrossentropy, MeanAbsoluteError
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Dropout, concatenate

print("All necessary packages installed")

All necessary packages installed


In [ ]:
def split_image_into_patches(image, mask, patch_size=256):
    height, width = image.shape[:2]
    num_patches_x = width // patch_size
    num_patches_y = height // patch_size

    image_patches = []
    mask_patches = []

    for i in range(num_patches_y):
        for j in range(num_patches_x):
            x_start = j * patch_size
            y_start = i * patch_size

            # Extract patches
            image_patch = image[y_start:y_start + patch_size, x_start:x_start + patch_size]
            mask_patch = mask[y_start:y_start + patch_size, x_start:x_start + patch_size]

            image_patches.append(image_patch)
            mask_patches.append(mask_patch)

    return image_patches, mask_patches


def load_data_with_patches(image_dir, mask_dir, patch_size=256, img_size=None):
    image_paths = sorted(glob.glob(os.path.join(image_dir, "image*.jpg")) +
                         glob.glob(os.path.join(image_dir, "image*.png")))

    print(f"Found {len(image_paths)} images in {image_dir}")
    all_image_patches = []
    all_mask_patches = []
    for img_path in image_paths:
        # Extract image number using the filename
        filename = os.path.basename(img_path)
        img_num = ''.join(filter(str.isdigit, filename))
        # Construct the corresponding mask filename
        mask_filename = f"mask{img_num}.png"
        mask_path = os.path.join(mask_dir, mask_filename)
        # Load image
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
        # Load mask
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        # Create patches
        image_patches, mask_patches = split_image_into_patches(img, mask, patch_size)
        # Process each patch
        for img_patch, mask_patch in zip(image_patches, mask_patches):
            # Normalize image to [0,1]
            img_patch = img_patch.astype(np.float32) / 255.0
            # Binarize the mask - assuming clouds are white/bright and background is dark
            mask_patch = (mask_patch > 128).astype(np.float32)  # Convert to binary (0 or 1)
            mask_patch = np.expand_dims(mask_patch, axis=-1)  # Add channel dimension
            all_image_patches.append(img_patch)
            all_mask_patches.append(mask_patch)
    # Convert to numpy arrays
    images = np.array(all_image_patches, dtype=np.float32)
    masks = np.array(all_mask_patches, dtype=np.float32)
    print(f"Created {len(images)} patches of size {patch_size}x{patch_size}")
    print(f"Images shape: {images.shape}")
    print(f"Masks shape: {masks.shape}")
    return images, masks


In [ ]:
def define_discriminator(image_shape=(256, 256, 3), mask_shape=(256, 256, 1)):
    # Weight initialization
    init = RandomNormal(stddev=0.02)  # As described in the original paper

    # Input for satellite image (RGB)
    in_satellite_image = Input(shape=image_shape, name='satellite_image')

    # Input for cloud mask (binary)
    in_cloud_mask = Input(shape=mask_shape, name='cloud_mask')

    # Concatenate image and mask channel-wise
    # This will create a 4-channel input (3 RGB + 1 mask)
    merged = Concatenate()([in_satellite_image, in_cloud_mask])

    # C64: 4x4 kernel Stride 2x2
    d = Conv2D(64, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)

    # C128: 4x4 kernel Stride 2x2
    d = Conv2D(128, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)

    # C256: 4x4 kernel Stride 2x2
    d = Conv2D(256, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)

    # C512: 4x4 kernel Stride 2x2
    # Not in the original paper. Comment this block if you want.
    d = Conv2D(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)

    # Second last output layer: 4x4 kernel but Stride 1x1
    d = Conv2D(512, (4, 4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)

    # Patch output
    d = Conv2D(1, (4, 4), padding='same', kernel_initializer=init)(d)
    patch_out = Activation('sigmoid')(d)

    # Define model
    model = Model([in_satellite_image, in_cloud_mask], patch_out)
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])

    return model


def prepare_discriminator_input(images, masks, generated_masks=None):
    if generated_masks is None:
        # For initial training or when using real masks
        return images, masks
    else:
        # For training with fake masks from generator
        return images, generated_masks


# # Example usage with loaded patches
# if __name__ == "__main__":
#     # Create discriminator
#     discriminator = define_discriminator()

#     # Print model summary
#     discriminator.summary()

    # For training with real data
    # real_labels = np.ones((batch_size, 16, 16, 1))  # PatchGAN output
    # d_loss_real = discriminator.train_on_batch([images, masks], real_labels)

    # For training with fake data
    # fake_masks = generator.predict(images)  # Generated masks
    # fake_labels = np.zeros((batch_size, 16, 16, 1))
    # d_loss_fake = discriminator.train_on_batch([images, fake_masks], fake_labels)

In [ ]:
# def define_encoder_block(layer_in, n_filters, batchnorm=True):
# 	# weight initialization
# 	init = RandomNormal(stddev=0.02)
# 	# add downsampling layer
# 	g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
# 	# conditionally add batch normalization
# 	if batchnorm:
# 		g = BatchNormalization()(g, training=True)
# 	# leaky relu activation
# 	g = LeakyReLU(alpha=0.2)(g)
# 	return g

# # define a decoder block to be used in generator
# def decoder_block(layer_in, skip_in, n_filters, dropout=True):
# 	# weight initialization
# 	init = RandomNormal(stddev=0.02)
# 	# add upsampling layer
# 	g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
# 	# add batch normalization
# 	g = BatchNormalization()(g, training=True)
# 	# conditionally add dropout
# 	if dropout:
# 		g = Dropout(0.5)(g, training=True)
# 	# merge with skip connection
# 	g = Concatenate()([g, skip_in])
# 	# relu activation
# 	g = Activation('relu')(g)
# 	return g

# # define the standalone generator model - U-net
# def define_generator(image_shape=(256,256,3)):
# 	# weight initialization
# 	init = RandomNormal(stddev=0.02)
# 	# image input
# 	in_image = Input(shape=image_shape)
# 	# encoder model: C64-C128-C256-C512-C512-C512-C512-C512
# 	e1 = define_encoder_block(in_image, 64, batchnorm=False)
# 	e2 = define_encoder_block(e1, 128)
# 	e3 = define_encoder_block(e2, 256)
# 	e4 = define_encoder_block(e3, 512)
# 	e5 = define_encoder_block(e4, 512)
# 	e6 = define_encoder_block(e5, 512)
# 	e7 = define_encoder_block(e6, 512)
# 	# bottleneck, no batch norm and relu
# 	b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
# 	b = Activation('relu')(b)
# 	# decoder model: CD512-CD512-CD512-C512-C256-C128-C64
# 	d1 = decoder_block(b, e7, 512)
# 	d2 = decoder_block(d1, e6, 512)
# 	d3 = decoder_block(d2, e5, 512)
# 	d4 = decoder_block(d3, e4, 512, dropout=False)
# 	d5 = decoder_block(d4, e3, 256, dropout=False)
# 	d6 = decoder_block(d5, e2, 128, dropout=False)
# 	d7 = decoder_block(d6, e1, 64, dropout=False)
# 	# output
# 	g = Conv2DTranspose(image_shape[2], (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7) #Modified
# 	out_image = Activation('tanh')(g)  #Generates images in the range -1 to 1. So change inputs also to -1 to 1
# 	# define model
# 	model = Model(in_image, out_image)
# 	return model



# def define_generator(image_shape=(256, 256, 3)):
#     # Weight initialization
#     init = RandomNormal(stddev=0.02)

#     # Image input
#     in_image = Input(shape=image_shape, name='satellite_image')

#     # Encoder
#     # C64
#     e1 = Conv2D(64, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(in_image)
#     e1 = LeakyReLU(alpha=0.2)(e1)

#     # C128
#     e2 = Conv2D(128, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(e1)
#     e2 = LeakyReLU(alpha=0.2)(e2)

#     # C256
#     e3 = Conv2D(256, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(e2)
#     e3 = LeakyReLU(alpha=0.2)(e3)

#     # C512
#     e4 = Conv2D(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(e3)
#     e4 = LeakyReLU(alpha=0.2)(e4)

#     # C512
#     e5 = Conv2D(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(e4)
#     e5 = LeakyReLU(alpha=0.2)(e5)

#     # C512
#     e6 = Conv2D(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(e5)
#     e6 = LeakyReLU(alpha=0.2)(e6)

#     # C512
#     e7 = Conv2D(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(e6)
#     e7 = LeakyReLU(alpha=0.2)(e7)

#     # Bottleneck
#     b = Conv2D(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(e7)
#     b = Activation('relu')(b)

#     # Decoder
#     # CD512
#     d1 = Conv2DTranspose(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(b)
#     d1 = Concatenate()([d1, e7])
#     d1 = Activation('relu')(d1)
#     d1 = Dropout(0.5)(d1)

#     # CD512
#     d2 = Conv2DTranspose(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d1)
#     d2 = Concatenate()([d2, e6])
#     d2 = Activation('relu')(d2)
#     d2 = Dropout(0.5)(d2)

#     # CD512
#     d3 = Conv2DTranspose(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d2)
#     d3 = Concatenate()([d3, e5])
#     d3 = Activation('relu')(d3)
#     d3 = Dropout(0.5)(d3)

#     # C512
#     d4 = Conv2DTranspose(512, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d3)
#     d4 = Concatenate()([d4, e4])
#     d4 = Activation('relu')(d4)

#     # C256
#     d5 = Conv2DTranspose(256, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d4)
#     d5 = Concatenate()([d5, e3])
#     d5 = Activation('relu')(d5)

#     # C128
#     d6 = Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d5)
#     d6 = Concatenate()([d6, e2])
#     d6 = Activation('relu')(d6)

#     # C64
#     d7 = Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d6)
#     d7 = Concatenate()([d7, e1])
#     d7 = Activation('relu')(d7)

#     # Output layer - single channel for binary mask
#     out_mask = Conv2DTranspose(1, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init)(d7)
#     out_mask = Activation('sigmoid')(out_mask)  # Sigmoid for binary mask

#     # Define model
#     model = Model(in_image, out_mask)

#     return model


def define_generator(image_shape=(256, 256, 3)):
    # Input layer
    in_image = Input(shape=image_shape, name='satellite_image')

    # Load VGG16 model as encoder (without top layers)
    vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=in_image)

    # Freeze VGG16 layers (optional - can be commented out if you want to fine-tune)
    for layer in vgg16.layers:
        layer.trainable = False

    # Create skip connections from VGG16 layers
    s1 = vgg16.get_layer("block1_conv2").output  # 256x256
    s2 = vgg16.get_layer("block2_conv2").output  # 128x128
    s3 = vgg16.get_layer("block3_conv3").output  # 64x64
    s4 = vgg16.get_layer("block4_conv3").output  # 32x32
    s5 = vgg16.get_layer("block5_conv3").output  # 16x16

    # Decoder path
    # Upsampling block 1 - from 16x16 to 32x32
    d1 = Conv2DTranspose(512, (3, 3), strides=(2, 2), padding='same', activation='relu')(s5)
    d1 = concatenate([d1, s4])
    d1 = Conv2D(512, (3, 3), padding='same', activation='relu')(d1)
    d1 = Conv2D(512, (3, 3), padding='same', activation='relu')(d1)
    d1 = Dropout(0.3)(d1)

    # Upsampling block 2 - from 32x32 to 64x64
    d2 = Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same', activation='relu')(d1)
    d2 = concatenate([d2, s3])
    d2 = Conv2D(256, (3, 3), padding='same', activation='relu')(d2)
    d2 = Conv2D(256, (3, 3), padding='same', activation='relu')(d2)
    d2 = Dropout(0.3)(d2)

    # Upsampling block 3 - from 64x64 to 128x128
    d3 = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same', activation='relu')(d2)
    d3 = concatenate([d3, s2])
    d3 = Conv2D(128, (3, 3), padding='same', activation='relu')(d3)
    d3 = Conv2D(128, (3, 3), padding='same', activation='relu')(d3)
    d3 = Dropout(0.3)(d3)

    # Upsampling block 4 - from 128x128 to 256x256
    d4 = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same', activation='relu')(d3)
    d4 = concatenate([d4, s1])
    d4 = Conv2D(64, (3, 3), padding='same', activation='relu')(d4)
    d4 = Conv2D(64, (3, 3), padding='same', activation='relu')(d4)

    # Output layer - adjusted for binary mask with sigmoid activation
    # Using 1 channel for binary segmentation
    out_mask = Conv2D(1, (1, 1), padding='same', activation='sigmoid')(d4)

    # Create model
    model = Model(inputs=in_image, outputs=out_mask)

    return model


In [ ]:
# # select a batch of random samples, returns images and target
# def generate_real_samples(dataset, n_samples, patch_shape):
# 	# unpack dataset
# 	trainA, trainB = dataset
# 	# choose random instances
# 	ix = randint(0, trainA.shape[0], n_samples)
# 	# retrieve selected images
# 	X1, X2 = trainA[ix], trainB[ix]
# 	# generate 'real' class labels (1)
# 	y = ones((n_samples, patch_shape, patch_shape, 1))
# 	return [X1, X2], y

# # generate a batch of images, returns images and targets
# def generate_fake_samples(g_model, samples, patch_shape):
# 	# generate fake instance
# 	X = g_model.predict(samples)
# 	# create 'fake' class labels (0)
# 	y = zeros((len(X), patch_shape, patch_shape, 1))
# 	return X, y

# # generate samples and save as a plot and save the model
# #GAN models do not converge, we just want to find a good balance between
# #the generator and the discriminator. Therefore, it makes sense to periodically
# #save the generator model and check how good the generated image looks.
# def summarize_performance(step, g_model, dataset, n_samples=3):
# 	# select a sample of input images
# 	[X_realA, X_realB], _ = generate_real_samples(dataset, n_samples, 1)
# 	# generate a batch of fake samples
# 	X_fakeB, _ = generate_fake_samples(g_model, X_realA, 1)
# 	# scale all pixels from [-1,1] to [0,1]
# 	X_realA = (X_realA + 1) / 2.0
# 	X_realB = (X_realB + 1) / 2.0
# 	X_fakeB = (X_fakeB + 1) / 2.0
# 	# plot real source images
# 	for i in range(n_samples):
# 		plt.subplot(3, n_samples, 1 + i)
# 		plt.axis('off')
# 		plt.imshow(X_realA[i])
# 	# plot generated target image
# 	for i in range(n_samples):
# 		plt.subplot(3, n_samples, 1 + n_samples + i)
# 		plt.axis('off')
# 		plt.imshow(X_fakeB[i])
# 	# plot real target image
# 	for i in range(n_samples):
# 		plt.subplot(3, n_samples, 1 + n_samples*2 + i)
# 		plt.axis('off')
# 		plt.imshow(X_realB[i])
# 	# save plot to file
# 	filename1 = 'plot_%06d.png' % (step+1)
# 	plt.savefig(filename1)
# 	plt.close()
# 	# save the generator model
# 	filename2 = 'model_%06d.h5' % (step+1)
# 	g_model.save(filename2)
# 	print('>Saved: %s and %s' % (filename1, filename2))

# # train pix2pix models
# def train(d_model, g_model, gan_model, dataset, n_epochs=100, n_batch=1):
# 	# determine the output square shape of the discriminator
# 	n_patch = d_model.output_shape[1]
# 	# unpack dataset
# 	trainA, trainB = dataset
# 	# calculate the number of batches per training epoch
# 	bat_per_epo = int(len(trainA) / n_batch)
# 	# calculate the number of training iterations
# 	n_steps = bat_per_epo * n_epochs
# 	# manually enumerate epochs
# 	for i in range(n_steps):
# 		# select a batch of real samples
# 		[X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
# 		# generate a batch of fake samples
# 		X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
# 		# update discriminator for real samples
# 		d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
# 		# update discriminator for generated samples
# 		d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
# 		# update the generator
# 		g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
# 		# summarize performance
# 		print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
# 		# summarize model performance
# 		if (i+1) % (bat_per_epo * 10) == 0:
# 			summarize_performance(i, g_model, dataset)


def generate_real_samples(dataset, n_samples, patch_shape):
    # Unpack dataset
    images, masks = dataset

    # Choose random instances
    ix = randint(0, images.shape[0], n_samples)

    # Retrieve selected images and masks
    X1, X2 = images[ix], masks[ix]

    # Generate 'real' class labels (1)
    y = ones((n_samples, patch_shape, patch_shape, 1))

    return [X1, X2], y

def generate_fake_samples(g_model, images, patch_shape):
    # Generate fake masks
    fake_masks = g_model.predict(images)

    # Create 'fake' class labels (0)
    y = zeros((len(fake_masks), patch_shape, patch_shape, 1))

    return fake_masks, y


def summarize_performance(step, g_model, dataset, n_samples=3):
    # Select a sample of input images
    [X_images, X_real_masks], _ = generate_real_samples(dataset, n_samples, 1)

    # Generate fake masks
    X_fake_masks = g_model.predict(X_images)

    # Plot real satellite images
    for i in range(n_samples):
        plt.subplot(3, n_samples, 1 + i)
        plt.axis('off')
        plt.imshow(X_images[i])
        plt.title('Satellite Image')

    # Plot generated masks
    for i in range(n_samples):
        plt.subplot(3, n_samples, 1 + n_samples + i)
        plt.axis('off')
        plt.imshow(X_fake_masks[i, :, :, 0], cmap='gray')
        plt.title('Generated Mask')

    # Plot real masks
    for i in range(n_samples):
        plt.subplot(3, n_samples, 1 + n_samples*2 + i)
        plt.axis('off')
        plt.imshow(X_real_masks[i, :, :, 0], cmap='gray')
        plt.title('Real Mask')

    # Save plot to file
    filename1 = 'cloud_mask_plot_%06d.png' % (step+1)
    plt.savefig(filename1, dpi=150, bbox_inches='tight')
    plt.close()

    # Save the generator model
    filename2 = 'cloud_mask_generator_%06d.h5' % (step+1)
    g_model.save(filename2)
    print('>Saved: %s and %s' % (filename1, filename2))


def train(d_model, g_model, gan_model, dataset, n_epochs=50, n_batch=1):
    # Determine the output square shape of the discriminator
    n_patch = d_model.output_shape[1]

    # Unpack dataset
    images, masks = dataset

    # Calculate the number of batches per training epoch
    bat_per_epo = int(len(images) / n_batch)

    # Calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs

    # Lists to store losses
    d_losses = []
    g_losses = []

    # Manually enumerate epochs
    for i in range(n_steps):
        # Select a batch of real samples
        [X_images, X_real_masks], y_real = generate_real_samples(dataset, n_batch, n_patch)

        # Generate a batch of fake samples
        X_fake_masks, y_fake = generate_fake_samples(g_model, X_images, n_patch)

        # Update discriminator for real samples
        d_loss1 = d_model.train_on_batch([X_images, X_real_masks], y_real)

        # Update discriminator for generated samples
        d_loss2 = d_model.train_on_batch([X_images, X_fake_masks], y_fake)

        # Update the generator
        g_loss = gan_model.train_on_batch(X_images, [y_real, X_real_masks])

        # Store losses
        d_losses.append((d_loss1, d_loss2))
        g_losses.append(g_loss)

        # Summarize performance
        print('>Step %d/%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, n_steps, d_loss1, d_loss2, g_loss[0]))

        # Summarize model performance periodically
        if (i+1) % (bat_per_epo * 10) == 0:
            summarize_performance(i, g_model, dataset)

    return d_losses, g_losses


In [ ]:
# # Example usage
# if __name__ == "__main__":
#     VAL_IMAGES_DIR = '/content/drive/MyDrive/SVM_Data/Train/Images'
#     VAL_MASKS_DIR = '/content/drive/MyDrive/SVM_Data/Train/Mask'
#     # Load data with patches
#     images, masks = load_data_with_patches(
#         image_dir=VAL_IMAGES_DIR,
#         mask_dir=VAL_MASKS_DIR,
#         patch_size=256
#     )
#     # Now you can use these patches for your GAN
#     # print(f"Ready for GAN training with {len(images)} patches")


def define_gan(g_model, d_model, image_shape=(256, 256, 3)):
    # Make weights in the discriminator not trainable
    for layer in d_model.layers:
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = False

    # Define the source image
    in_src = Input(shape=image_shape)

    # Generate mask from source image
    gen_mask = g_model(in_src)

    # Connect the source image and generated mask as input to discriminator
    dis_out = d_model([in_src, gen_mask])

    # Define combined model
    model = Model(in_src, [dis_out, gen_mask])

    # Compile model
    # Use both adversarial loss and L1 loss
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(
        loss=['binary_crossentropy', 'mae'],  # Binary crossentropy for discriminator, MAE for pixel-wise loss
        optimizer=opt,
        loss_weights=[1, 100]  # Weight pixel loss more heavily
    )

    return model

# Main function with all components
if __name__ == "__main__":
    # Define directories
    VAL_IMAGES_DIR = '/content/drive/MyDrive/SVM_Data/Train/Images'
    VAL_MASKS_DIR = '/content/drive/MyDrive/SVM_Data/Train/Mask'

    # Create output directory for saving models and plots
    output_dir = '/content/drive/MyDrive/cloud_mask_gan_output'
    os.makedirs(output_dir, exist_ok=True)

    # Change working directory to output directory
    os.chdir(output_dir)

    print("Loading data with patches...")
    # Load data with patches
    images, masks = load_data_with_patches(
        image_dir=VAL_IMAGES_DIR,
        mask_dir=VAL_MASKS_DIR,
        patch_size=256
    )

    print(f"Loaded {len(images)} patches of size 256x256")
    print(f"Images shape: {images.shape}")
    print(f"Masks shape: {masks.shape}")

    # Create dataset tuple
    dataset = (images, masks)

    # Define input shape
    image_shape = (256, 256, 3)
    mask_shape = (256, 256, 1)

    print("\nCreating models...")
    # Create discriminator
    d_model = define_discriminator(image_shape=image_shape, mask_shape=mask_shape)
    print("Discriminator created")

    # Create generator
    g_model = define_generator(image_shape=image_shape)
    print("Generator created")

    # Create GAN
    gan_model = define_gan(g_model, d_model, image_shape=image_shape)
    print("GAN model created")

    # Print model summaries
    print("\n=== Discriminator Summary ===")
    d_model.summary()

    print("\n=== Generator Summary ===")
    g_model.summary()

    print("\n=== GAN Summary ===")
    gan_model.summary()

    # Training parameters
    n_epochs = 200
    n_batch = 8  # Adjust based on your GPU memory

    print(f"\nStarting training for {n_epochs} epochs with batch size {n_batch}...")

    # Train the models
    try:
        d_losses, g_losses = train(
            d_model=d_model,
            g_model=g_model,
            gan_model=gan_model,
            dataset=dataset,
            n_epochs=n_epochs,
            n_batch=n_batch
        )

        print("\nTraining completed successfully!")

        # Save final models
        print("Saving final models...")
        g_model.save('final_generator.h5')
        d_model.save('final_discriminator.h5')
        print("Models saved!")

        # Plot training history
        plt.figure(figsize=(15, 5))

        # Plot discriminator losses
        plt.subplot(1, 2, 1)
        d_loss_real = [loss[0] for loss in d_losses]
        d_loss_fake = [loss[1] for loss in d_losses]
        plt.plot(d_loss_real, label='D Loss Real')
        plt.plot(d_loss_fake, label='D Loss Fake')
        plt.title('Discriminator Losses')
        plt.xlabel('Training Steps')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)

        # Plot generator losses
        plt.subplot(1, 2, 2)
        g_loss_total = [loss[0] for loss in g_losses]
        plt.plot(g_loss_total, label='G Loss Total')
        plt.title('Generator Loss')
        plt.xlabel('Training Steps')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)

        plt.tight_layout()
        plt.savefig('training_history.png', dpi=300)
        plt.close()

        print("Training history plot saved!")

        # Generate some final samples for evaluation
        print("\nGenerating final evaluation samples...")
        n_samples = 5
        sample_indices = np.random.randint(0, len(images), n_samples)
        sample_images = images[sample_indices]
        sample_real_masks = masks[sample_indices]
        sample_fake_masks = g_model.predict(sample_images)

        # Plot final results
        fig, axes = plt.subplots(3, n_samples, figsize=(15, 9))

        for i in range(n_samples):
            # Plot satellite image
            axes[0, i].imshow(sample_images[i])
            axes[0, i].set_title('Satellite Image')
            axes[0, i].axis('off')

            # Plot generated mask
            axes[1, i].imshow(sample_fake_masks[i, :, :, 0], cmap='gray', vmin=0, vmax=1)
            axes[1, i].set_title('Generated Mask')
            axes[1, i].axis('off')

            # Plot real mask
            axes[2, i].imshow(sample_real_masks[i, :, :, 0], cmap='gray', vmin=0, vmax=1)
            axes[2, i].set_title('Real Mask')
            axes[2, i].axis('off')

        plt.tight_layout()
        plt.savefig('final_results.png', dpi=300, bbox_inches='tight')
        plt.close()

        print("Final results plot saved!")
        print("\nAll outputs saved to:", output_dir)

    except Exception as e:
        print(f"\nError during training: {e}")
        # Save models even if training is interrupted
        print("Saving current models...")
        g_model.save('interrupted_generator.h5')
        d_model.save('interrupted_discriminator.h5')
        print("Models saved!")
        raise e

Loading data with patches...
Found 5 images in /content/drive/MyDrive/SVM_Data/Train/Images
Created 45 patches of size 256x256
Images shape: (45, 256, 256, 3)
Masks shape: (45, 256, 256, 1)
Loaded 45 patches of size 256x256
Images shape: (45, 256, 256, 3)
Masks shape: (45, 256, 256, 1)

Creating models...
Discriminator created
Generator created
GAN model created

=== Discriminator Summary ===


Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ satellite_image     │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cloud_mask          │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_5       │ (None, 256, 256,  │          0 │ satellite_image[… │
│ (Concatenate)       │ 4)                │            │ cloud_mask[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_15 (Conv2D)  │ (None, 128, 128,  │      4,160 │ concatenate_5[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_5       │ (None, 128, 128,  │          0 │ conv2d_15[0][0]   │
│ (LeakyReLU)         │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_16 (Conv2D)  │ (None, 64, 64,    │    131,200 │ leaky_re_lu_5[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        512 │ conv2d_16[0][0]   │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_6       │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (LeakyReLU)         │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_17 (Conv2D)  │ (None, 32, 32,    │    524,544 │ leaky_re_lu_6[0]… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 32,    │      1,024 │ conv2d_17[0][0]   │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_7       │ (None, 32, 32,    │          0 │ batch_normalizat… │
│ (LeakyReLU)         │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_18 (Conv2D)  │ (None, 16, 16,    │  2,097,664 │ leaky_re_lu_7[0]… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16,    │      2,048 │ conv2d_18[0][0]   │
│ (BatchNormalizatio… │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_8       │ (None, 16, 16,    │          0 │ batch_normalizat… │
│ (LeakyReLU)         │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_19 (Conv2D)  │ (None, 16, 16,    │  4,194,816 │ leaky_re_lu_8[0]… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16, 16,    │      2,048 │ conv2d_19[0][0]   │
│ (BatchNormalizatio… │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ leaky_re_lu_9       │ (None, 16, 16,    │          0 │ batch_normalizat

 Total params: 6,966,209 (26.57 MB)

 Trainable params: 2,816 (11.00 KB)

...

>Saved: cloud_mask_plot_000850.png and cloud_mask_generator_000850.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
>Step 851/1000, d1[0.414] d2[0.414] g[5.061]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
>Step 852/1000, d1[0.414] d2[0.414] g[5.058]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 215ms/step
>Step 853/1000, d1[0.414] d2[0.414] g[5.053]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
>Step 854/1000, d1[0.414] d2[0.414] g[5.049]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
>Step 855/1000, d1[0.414] d2[0.414] g[5.045]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
>Step 856/1000, d1[0.414] d2[0.414] g[5.040]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
>Step 857/1000, d1[0.414] d2[0.414] g[5.035]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
>Step 858/1000, d1[0.414] d2[0.414] g[5.031]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
>Step 859/1000, d1[0.414] d2[0.414] g[5.026]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
>Step 860/1000, d1[0.414] d2[0.414] g[5.022]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
>Step 861/1000, d1[0.414] d2[0.414] g[5.019]
1/1 ━━

>Saved: cloud_mask_plot_000900.png and cloud_mask_generator_000900.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step
>Step 901/1000, d1[0.415] d2[0.415] g[4.862]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 205ms/step
>Step 902/1000, d1[0.415] d2[0.415] g[4.859]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
>Step 903/1000, d1[0.415] d2[0.415] g[4.855]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
>Step 904/1000, d1[0.415] d2[0.415] g[4.852]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
>Step 905/1000, d1[0.415] d2[0.415] g[4.848]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
>Step 906/1000, d1[0.415] d2[0.415] g[4.845]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
>Step 907/1000, d1[0.415] d2[0.415] g[4.841]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
>Step 908/1000, d1[0.415] d2[0.415] g[4.837]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
>Step 909/1000, d1[0.415] d2[0.415] g[4.833]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
>Step 910/1000, d1[0.415] d2[0.415] g[4.829]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
>Step 911/1000, d1[0.415] d2[0.415] g[4.825]
1/1 ━━

>Saved: cloud_mask_plot_000950.png and cloud_mask_generator_000950.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
>Step 951/1000, d1[0.415] d2[0.415] g[4.686]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 201ms/step
>Step 952/1000, d1[0.415] d2[0.415] g[4.683]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
>Step 953/1000, d1[0.415] d2[0.415] g[4.679]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
>Step 954/1000, d1[0.415] d2[0.415] g[4.676]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
>Step 955/1000, d1[0.415] d2[0.415] g[4.672]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
>Step 956/1000, d1[0.415] d2[0.415] g[4.669]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
>Step 957/1000, d1[0.415] d2[0.415] g[4.665]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
>Step 958/1000, d1[0.415] d2[0.415] g[4.661]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
>Step 959/1000, d1[0.415] d2[0.415] g[4.658]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step
>Step 960/1000, d1[0.415] d2[0.416] g[4.655]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
>Step 961/1000, d1[0.415] d2[0.416] g[4.653]
1/1 ━━

>Saved: cloud_mask_plot_001000.png and cloud_mask_generator_001000.h5

Training completed successfully!
Saving final models...


Models saved!
Training history plot saved!

Generating final evaluation samples...
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step
Final results plot saved!

All outputs saved to: /content/drive/MyDrive/cloud_mask_gan_output
